In [ ]:
from datetime import datetime

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from db_utils import get_rect_to_rect_data

In [ ]:
ped_files = ['VM2_-2112701535', 'VM2_-217686115','VM2_-1247665811','VM2_-104300786', 'VM2_330973206', 'VM2_1100569031', 'VM2_421371629', 'VM2_421371629', 'VM2_1476499235', 'VM2_-1451152685', 'VM2_-1523872256']

In [ ]:
start_rect_coords = (13.416257,52.521664,13.416462,52.521793)
end_rect_coords = (13.416026,52.522284,13.416231,52.522413)

end_date_str = '2021-01-01 00:00:00'
end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M:%S')

df_simra = get_rect_to_rect_data(start_rect_coords, end_rect_coords, end_date=end_date, files_to_exclude=ped_files)
# df_simra = get_rect_to_rect_data(start_rect_coords, end_rect_coords, end_date=end_date, files_to_exclude=None)

In [ ]:
# Some entries contain nans, or no speed, even though a distance is given. Inspect further. Option for filtering or preprocessing.


# df_simra.loc[(df_simra==0).any(axis=1)]
# df_simra.loc[(df_simra.isna()).any(axis=1)]

# df_simra[(df_simra.velo == 0) | (df_simra.velo.isna())]
df_simra[((df_simra.velo == 0) | (df_simra.velo.isna())) & (df_simra.dist != 0.0)]

In [ ]:
# df_simra.loc[df_simra['filename'] == df_simra['filename'][27]]

In [ ]:
df_simra

In [ ]:
df_simra_grouped = df_simra.groupby('filename').agg({'velo': 'mean', 'dist': 'sum'})
# df_simra_grouped

In [ ]:
# # rotate vector by 90 degrees clockwise
# #    (x, y)     -> (y, -x)
# # =  (lon, lat) -> (lat, -lon)
# lat_start = df_simra.groupby('filename').first()['lat'].rename('start_lat')
# lat_end = df_simra.groupby('filename').last()['lat']

# delta_lat = lat_end - lat_start

# lon_start = df_simra.groupby('filename').first()['lon'].rename('start_lon')
# lon_end = df_simra.groupby('filename').last()['lon']

# delta_lon = lon_end - lon_start

# path_rotated_lon_unnormalized = delta_lat
# path_rotated_lat_unnormalized = (-1) *  delta_lon

# path_rotated_lon = path_rotated_lon_unnormalized / np.sqrt(path_rotated_lon_unnormalized**2+ path_rotated_lat_unnormalized**2)
# path_rotated_lat = path_rotated_lat_unnormalized / np.sqrt(path_rotated_lon_unnormalized**2+ path_rotated_lat_unnormalized**2)

# path_rotated_lat = path_rotated_lat.rename('path_rotated_lat')
# path_rotated_lon = path_rotated_lon.rename('path_rotated_lon')

In [ ]:
# rotate vector by 90 degrees clockwise
#    (x, y)     -> (y, -x)
# =  (lon, lat) -> (lat, -lon)
lat_start = df_simra.iloc[0].lat
lat_end = df_simra.iloc[-1].lat

delta_lat = lat_end - lat_start

lon_start = df_simra.iloc[0].lat
lon_end = df_simra.iloc[-1].lat

delta_lon = lon_end - lon_start

path_rotated_lon_unnormalized = delta_lat
path_rotated_lat_unnormalized = (-1) *  delta_lon

path_rotated_lon = path_rotated_lon_unnormalized / np.sqrt(path_rotated_lon_unnormalized**2+ path_rotated_lat_unnormalized**2)
path_rotated_lat = path_rotated_lat_unnormalized / np.sqrt(path_rotated_lon_unnormalized**2+ path_rotated_lat_unnormalized**2)

In [ ]:
path_rotated_lon

In [ ]:
df_simra_grouped_vec = pd.concat([df_simra_grouped.reset_index()], axis =1)
# df_simra_grouped_vec

In [ ]:
max_projections = []
for filename in df_simra_grouped_vec['filename']:
    print(filename)
    projections = []
    x = path_rotated_lat
    y = path_rotated_lon
    x_norm = x / np.sqrt(x**2 + y**2)
    y_norm = y / np.sqrt(x**2 + y**2)
    print(f"x: {x}\n y: {y}")
    print(f"x: {x_norm}\n y: {y_norm}\n")
    for lat, lon in zip(df_simra[df_simra['filename'] == filename]['lat'], df_simra[df_simra['filename'] == filename]['lon']):
        print(f"lat: {lat}, lon: {lon}")
        projection = (lat * x_norm) + (lon * y_norm)
        print(f'projection: {projection}')
        projections.append(projection)
    print('projections:\n', projections)
    max_projections.append(np.max(projections))

df_simra_grouped_vec['max_projection'] = max_projections


    # print(projections)
    # print(f"max {np.max(projections)}")

    # print(f"min {np.min(projections)}") #just to check, but it should be only the max

In [ ]:
df_simra_grouped_vec

In [ ]:
# df_simra_grouped_vec.loc[df_simra_grouped_vec['filename']==df_simra_grouped_vec['filename'][0]]['path_rotated_lat']

In [ ]:
plt.hist(max_projections, bins = 50);

In [ ]:
# Scale distances and velocities with min-max-scaler to make distances comparable
Scaler = MinMaxScaler()

dists_unscaled = np.array(df_simra_grouped.dist).reshape(-1, 1)
velos_unscaled = np.array(df_simra_grouped.velo).reshape(-1, 1)
max_projections_unscaled = np.array(max_projections).reshape(-1, 1)

dists = Scaler.fit_transform(dists_unscaled)
velos = Scaler.fit_transform(velos_unscaled)
max_projections = Scaler.fit_transform(max_projections_unscaled)


### Review: Velocity depened graph

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.collections import LineCollection

fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect('equal')
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(df_simra.velo.min(), df_simra.velo.max())

df_simra_grouped = df_simra.groupby('filename')
for ride_group_name in df_simra_grouped.groups:
    df_ride_group = df_simra_grouped.get_group(ride_group_name)
    points = np.array([df_ride_group.lon, df_ride_group.lat]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments, cmap='viridis', norm=norm)
    # Set the values used for colormapping
    lc.set_array(df_ride_group.velo.values)
    lc.set_linewidth(2)
    line = ax.add_collection(lc)

ax.set_xlim(min(df_simra.lon), max(df_simra.lon))
ax.set_ylim(min(df_simra.lat), max(df_simra.lat))

axcb = fig.colorbar(lc, ax=ax, shrink=0.8, pad=0.1)
axcb.set_label('Velocity in m/s')

ax.xaxis.set_major_locator(ticker.LinearLocator(4))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.yaxis.set_major_locator(ticker.LinearLocator(4))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.set_xlabel('Longitude in decimal degrees')
ax.set_ylabel('Latitude in decimal degrees')

plt.show()

### Average Velocity & Distance

In [ ]:
N_CLUSTERS = 2
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0)

print(velos.shape)
print(dists.shape)

velo_dist_combined = np.hstack([velos, dists])
print(velo_dist_combined.shape)

velo_dist_cluster_labels = kmeans.fit_predict(velo_dist_combined)
velo_dist_cluster_centers = kmeans.cluster_centers_

colors = ['blue' if label == 0 else 'orange' for label in velo_dist_cluster_labels]
plt.scatter(velo_dist_combined[:,0], velo_dist_combined[:,1], c=colors)
plt.scatter(velo_dist_cluster_centers[:,0], velo_dist_cluster_centers[:,1], color='red')
plt.xlabel('velocity (min-max-scaled)')
plt.ylabel('distance (min-max-scaled)')


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.collections import LineCollection

fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect('equal')
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(df_simra.velo.min(), df_simra.velo.max())

df_simra_grouped = df_simra.groupby('filename')
for i, ride_group_name in enumerate(df_simra_grouped.groups):
    df_ride_group = df_simra_grouped.get_group(ride_group_name)
    points = np.array([df_ride_group.lon, df_ride_group.lat]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments)
    # Set the color according to cluster
    lc.set_linewidth(2)
    if velo_dist_cluster_labels[i] == 0:
        col = 'blue'
    else:
        col = 'orange'
    lc.set_edgecolors(col)
    line = ax.add_collection(lc)

ax.set_xlim(min(df_simra.lon), max(df_simra.lon))
ax.set_ylim(min(df_simra.lat), max(df_simra.lat))

# axcb = fig.colorbar(lc, ax=ax, shrink=0.8, pad=0.1)
# axcb.set_label('Velocity in m/s')

ax.xaxis.set_major_locator(ticker.LinearLocator(4))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.yaxis.set_major_locator(ticker.LinearLocator(4))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.set_xlabel('Longitude in decimal degrees')
ax.set_ylabel('Latitude in decimal degrees')


plt.show()

### Average Velocity

In [ ]:
N_CLUSTERS = 2
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0)


velo_cluster_labels = kmeans.fit_predict(velos)
velo_cluster_centers = kmeans.cluster_centers_

sns.stripplot(x=[x_ for x_, label in zip(velos, velo_cluster_labels) if label == 0], color='blue')
sns.stripplot(x=[x_ for x_, label in zip(velos, velo_cluster_labels) if label == 1], color='orange')
sns.stripplot(x=velo_cluster_centers, color='red', size=10, jitter=False)
plt.xlabel('velocities (min-max-scaled)')


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.collections import LineCollection

fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect('equal')
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(df_simra.velo.min(), df_simra.velo.max())

df_simra_grouped = df_simra.groupby('filename')
for i, ride_group_name in enumerate(df_simra_grouped.groups):
    df_ride_group = df_simra_grouped.get_group(ride_group_name)
    points = np.array([df_ride_group.lon, df_ride_group.lat]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments)
    # Set the color according to cluster
    lc.set_linewidth(2)
    if velo_cluster_labels[i] == 0:
        col = 'blue'
    else:
        col = 'orange'
    lc.set_edgecolors(col)
    line = ax.add_collection(lc)

ax.set_xlim(min(df_simra.lon), max(df_simra.lon))
ax.set_ylim(min(df_simra.lat), max(df_simra.lat))

# axcb = fig.colorbar(lc, ax=ax, shrink=0.8, pad=0.1)
# axcb.set_label('Velocity in m/s')

ax.xaxis.set_major_locator(ticker.LinearLocator(4))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.yaxis.set_major_locator(ticker.LinearLocator(4))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.set_xlabel('Longitude in decimal degrees')
ax.set_ylabel('Latitude in decimal degrees')


plt.show()

### Distance

In [ ]:
N_CLUSTERS = 2
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0)

dist_cluster_labels = kmeans.fit_predict(dists)
dist_cluster_centers = kmeans.cluster_centers_

sns.stripplot(x=[x_ for x_, label in zip(dists, dist_cluster_labels) if label == 0], color='blue')
sns.stripplot(x=[x_ for x_, label in zip(dists, dist_cluster_labels) if label == 1], color='orange')
sns.stripplot(x=dist_cluster_centers, color='red', size=10, jitter=False)
plt.xlabel('distance (min-max-scaled)')


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.collections import LineCollection

fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect('equal')
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(df_simra.velo.min(), df_simra.velo.max())

df_simra_grouped = df_simra.groupby('filename')
for i, ride_group_name in enumerate(df_simra_grouped.groups):
    df_ride_group = df_simra_grouped.get_group(ride_group_name)
    points = np.array([df_ride_group.lon, df_ride_group.lat]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments)
    # Set the color according to cluster
    lc.set_linewidth(2)
    if dist_cluster_labels[i] == 0:
        col = 'blue'
    else:
        col = 'orange'
    lc.set_edgecolors(col)
    line = ax.add_collection(lc)

ax.set_xlim(min(df_simra.lon), max(df_simra.lon))
ax.set_ylim(min(df_simra.lat), max(df_simra.lat))

# axcb = fig.colorbar(lc, ax=ax, shrink=0.8, pad=0.1)
# axcb.set_label('Velocity in m/s')

ax.xaxis.set_major_locator(ticker.LinearLocator(4))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.yaxis.set_major_locator(ticker.LinearLocator(4))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.set_xlabel('Longitude in decimal degrees')
ax.set_ylabel('Latitude in decimal degrees')


plt.show()

### Distance & Projection

In [ ]:
N_CLUSTERS = 2
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0)

print(velos.shape)
print(dists.shape)

projection_dist_combined = np.hstack([max_projections, dists])
print(projection_dist_combined.shape)

projection_dist_cluster_labels = kmeans.fit_predict(projection_dist_combined)
projection_dist_cluster_centers = kmeans.cluster_centers_

colors = ['blue' if label == 0 else 'orange' for label in projection_dist_cluster_labels]
plt.scatter(projection_dist_combined[:,0], projection_dist_combined[:,1], c=colors)
plt.scatter(projection_dist_cluster_centers[:,0], projection_dist_cluster_centers[:,1], color='red')
plt.xlabel('max_projection (min-max-scaled)')
plt.ylabel('distance (min-max-scaled)')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.collections import LineCollection

fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect('equal')
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(df_simra.velo.min(), df_simra.velo.max())

df_simra_grouped = df_simra.groupby('filename')
for i, ride_group_name in enumerate(df_simra_grouped.groups):
    df_ride_group = df_simra_grouped.get_group(ride_group_name)
    points = np.array([df_ride_group.lon, df_ride_group.lat]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments)
    # Set the color according to cluster
    lc.set_linewidth(2)
    if projection_dist_cluster_labels[i] == 0:
        col = 'blue'
    else:
        col = 'orange'
    lc.set_edgecolors(col)
    line = ax.add_collection(lc)

ax.set_xlim(min(df_simra.lon), max(df_simra.lon))
ax.set_ylim(min(df_simra.lat), max(df_simra.lat))

# axcb = fig.colorbar(lc, ax=ax, shrink=0.8, pad=0.1)
# axcb.set_label('Velocity in m/s')

ax.xaxis.set_major_locator(ticker.LinearLocator(4))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.yaxis.set_major_locator(ticker.LinearLocator(4))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.set_xlabel('Longitude in decimal degrees')
ax.set_ylabel('Latitude in decimal degrees')


plt.show()

### Projection

In [ ]:
N_CLUSTERS = 2
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0)

max_projections_cluster_labels = kmeans.fit_predict(max_projections)
max_projections_cluster_centers = kmeans.cluster_centers_

sns.stripplot(x=[x_ for x_, label in zip(max_projections, max_projections_cluster_labels) if label == 0], color='blue')
sns.stripplot(x=[x_ for x_, label in zip(max_projections, max_projections_cluster_labels) if label == 1], color='orange')
sns.stripplot(x=max_projections_cluster_centers, color='red', size=10, jitter=False)
plt.xlabel('max_projection (min-max-scaled)')


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.collections import LineCollection

fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect('equal')
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(df_simra.velo.min(), df_simra.velo.max())

df_simra_grouped = df_simra.groupby('filename')
for i, ride_group_name in enumerate(df_simra_grouped.groups):
    df_ride_group = df_simra_grouped.get_group(ride_group_name)
    points = np.array([df_ride_group.lon, df_ride_group.lat]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments)
    # Set the color according to cluster
    lc.set_linewidth(2)
    if max_projections_cluster_labels[i] == 0:
        col = 'blue'
    else:
        col = 'orange'
    lc.set_edgecolors(col)
    line = ax.add_collection(lc)

ax.set_xlim(min(df_simra.lon), max(df_simra.lon))
ax.set_ylim(min(df_simra.lat), max(df_simra.lat))

# axcb = fig.colorbar(lc, ax=ax, shrink=0.8, pad=0.1)
# axcb.set_label('Velocity in m/s')

ax.xaxis.set_major_locator(ticker.LinearLocator(4))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.yaxis.set_major_locator(ticker.LinearLocator(4))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.set_xlabel('Longitude in decimal degrees')
ax.set_ylabel('Latitude in decimal degrees')


plt.show()

In [ ]:
df_simra_grouped_vec['max_projection']

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker

fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect('equal')

colors = ['blue', 'orange']

df_simra_grouped = df_simra.groupby('filename')
for i, ride_group_name in enumerate(df_simra_grouped.groups):
    # if i == 8: break
    df_ride_group = df_simra_grouped.get_group(ride_group_name)
    ax.plot(df_ride_group.lon, df_ride_group.lat, color=colors[max_projections_cluster_labels[i]])

    df_ride_group_vec = df_simra_grouped_vec[df_simra_grouped_vec.filename == ride_group_name]
    vec_rot_lon = [lon_start, lon_start + path_rotated_lon]
    vec_rot_lat = [lat_start, lat_start + path_rotated_lat]
    ax.plot(vec_rot_lon, vec_rot_lat, color='green')

    projection_point_lon = np.float(path_rotated_lon) * df_ride_group_vec['max_projection']
    projection_point_lat = np.float(path_rotated_lat) * df_ride_group_vec['max_projection']

    print(projection_point_lon, projection_point_lat)
    ax.scatter([projection_point_lon], [projection_point_lat], color='red', s=50)


ax.set_xlim(min(df_simra.lon), max(df_simra.lon))
ax.set_ylim(min(df_simra.lat), max(df_simra.lat))

ax.xaxis.set_major_locator(ticker.LinearLocator(4))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.yaxis.set_major_locator(ticker.LinearLocator(4))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.set_xlabel('Longitude in decimal degrees')
ax.set_ylabel('Latitude in decimal degrees')


plt.show()

In [ ]:
plt.hist(max_projections)